# Aufgabe 1 & 2

In [22]:
import numpy as np
from tqdm import tqdm
from time import sleep
import operator
import copy

In [23]:
class NN():
    """A class to build and train a neural network used to solve the XOR problem."""
    INPUT_NEURONS = 2
    OUTPUT_NEURONS = 1

    def __init__(self, hidden_neurons=2, pop_size=10):
        """Create a 3-layer NN with two input neurons, n hidden neurons and one output neuron.
        
        hidden_neurons -- the number of neurons in the NN's hidden layer
        pop_size       -- the initial number of individuals in a population
        """
        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.h_neurons = hidden_neurons

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(NN.INPUT_NEURONS, self.h_neurons)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.h_neurons, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1, self.h_neurons)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Initial weights: ")
        self.print_weights()

    def ReLU(self, x):
        """ReLU activation function"""
        return x * (x > 0) 
    
    def sigmoid(self, x):
        """Sigmoid activation function"""
        return 1/(1 + np.exp(-x))
    
    def BCEL(self, y, y_hat):
        """Binary Cross-Entropy Loss
        
        y     -- the ground truth
        y_hat -- the NN's prediction
        """
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def execute(self, x):
        """Execute the NN for a given input, return its result."""

        # For the execution, the weights of the first (index 0) individual in the population are used.
        result = self.sigmoid(self.forward_pass(0, x)[0])
        return np.round(result[0])
    
    def forward_pass(self, index, x):
        """Carry out the forward pass.
        
        index -- for using the weights of one specific individual of the population
        """
        x = np.array(x)
        z0 = np.dot(x, self.w_hidden[index]) + self.b_hidden[index]
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden, self.w_out[index]) + self.b_out[index]
        return z1, z0


    def neuro_train(self, print_details = True, crossover = False):
        """Train the NN's weights using evolutionary algorithms."""
        epoch = 1
        # The weights of the first (index 0) individual in the population are used to check the NN's accuracy.
        while self.accuracy(0)[0] != 1:
            if print_details: print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))

            w_out = self.copy(self.w_out)
            b_out = self.copy(self.b_out)
            w_hidden = self.copy(self.w_hidden)
            b_hidden = self.copy(self.b_hidden)
            
            if crossover:
                w_out = self.crossover(w_out)
                b_out = self.crossover(b_out)
                w_hidden = self.crossover(w_hidden)
                b_hidden = self.crossover(b_hidden)

            self.mutation(w_out)
            self.mutation(b_out)
            self.mutation(w_hidden)
            self.mutation(b_hidden)
            
            self.w_out += w_out
            self.w_hidden += w_hidden
            self.b_out += b_out
            self.b_hidden += b_hidden

            f_list = self.fitness()
            self.selection(f_list)

            epoch += 1

        print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))
        self.print_weights()
        return epoch
    
    def copy(self, weight):
        """Create a copy of a given weight (list)."""
        new_weight = []
        for w in weight:
            new_weight.append(w.copy())
        return new_weight
    
    def fitness(self):
        """Calculate the fitness of each individual in the population, using the respective loss."""
        f_list = []
        for i in range(len(self.w_hidden)):
            acc, loss = self.accuracy(i)
            f_list += [1-acc+loss]
        return f_list

    def selection(self, f_list, use_prop_slct = False):
        """Select a subset of the population to be kept.
        
        f_list        -- list of fitness values for every individual
        use_prop_slct -- whether to use fitness proportional selection (default False)
        """
        dic = dict()
        for i in range(len(f_list)):
            dic.update([(i,f_list[i])])
        sortedPop = sorted(dic.items(), key=operator.itemgetter(1), reverse=False)
        last = sortedPop[self.pop_size:]
        last.sort()
        c = 0
        for index in last:
            del self.w_out[index[0]-c]
            del self.w_hidden[index[0]-c]
            del self.b_out[index[0]-c]
            del self.b_hidden[index[0]-c]
            c += 1
        

    def crossover(self, weight):
        """Perform a crossover on a given weight (list)."""
        w = np.zeros(shape=(len(weight), weight[0].shape[0], weight[0].shape[1]))
        lis = [i for i in range(len(weight))]
        np.random.shuffle(lis)
        for k in range(0, len(weight), 2):
            # prevent overflow in case of odd number of individuals
            if len(weight) % 2 != 0 and k == len(weight)-1: break
            for i in range(0, weight[0].shape[0]):
                for j in range(0, weight[0].shape[1]):
                    if np.random.sample() > 0.5:
                        w[lis[k]][i][j] = weight[lis[k+1]][i][j]
                        w[lis[k+1]][i][j] = weight[lis[k]][i][j]
                    else:
                        w[lis[k]][i][j] = weight[lis[k]][i][j]
                        w[lis[k+1]][i][j] = weight[lis[k+1]][i][j]
        return list(w)

    def mutation(self, weight):
        """Mutate a given weight (list) by adding values sampled from a normal destribution onto each entry."""
        for k in range(len(weight)):
            for i in range(len(weight[k])):
                for j in range(len(weight[k][i])):
                    weight[k][i][j] += np.random.normal(0,0.1)


    def print_weights(self):
        """Print the weights of the NN."""

        print("W_hidden: " + str(self.w_hidden[0])) 
        print("B_hidden: " + str(self.b_hidden[0])) 
        print("W_out: " + str(self.w_out[0])) 
        print("B_out: " + str(self.b_out[0]), end="\n\n")
        pass

    def accuracy(self, index):
        """Calculate the accuracy and loss of the NN for a specific individual.
        
        index -- for using the weights of one specific individual of the population
        """
        counter = 0
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        
        loss = 0
        for i in range(len(self.data)):
            loss += self.BCEL(self.target[i], self.sigmoid(self.forward_pass(index, self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [24]:
nn = NN(4, pop_size=20)

Initial weights: 
W_hidden: [[0.37038779 0.27163035 0.05058051 0.89366474]
 [0.37714054 0.58508205 0.41311001 0.63019695]]
B_hidden: [[0.60448324 0.65610096 0.40702213 0.70031398]]
W_out: [[0.81755474]
 [0.69755089]
 [0.16805784]
 [0.69689012]]
B_out: [[0.71950867]]



In [25]:
nn.neuro_train(print_details=True, crossover=True)

Epoch: 1 current acc: 0.5
Epoch: 2 current acc: 0.5
Epoch: 3 current acc: 0.5
Epoch: 4 current acc: 0.5
Epoch: 5 current acc: 0.5
Epoch: 6 current acc: 0.5
Epoch: 7 current acc: 0.5
Epoch: 8 current acc: 0.5
Epoch: 9 current acc: 0.5
Epoch: 10 current acc: 0.5
Epoch: 11 current acc: 0.5
Epoch: 12 current acc: 0.5
Epoch: 13 current acc: 0.5
Epoch: 14 current acc: 0.5
Epoch: 15 current acc: 0.5
Epoch: 16 current acc: 0.75
Epoch: 17 current acc: 0.75
Epoch: 18 current acc: 0.75
Epoch: 19 current acc: 0.75
Epoch: 20 current acc: 0.75
Epoch: 21 current acc: 0.75
Epoch: 22 current acc: 0.5
Epoch: 23 current acc: 0.5
Epoch: 24 current acc: 0.5
Epoch: 25 current acc: 0.5
Epoch: 26 current acc: 0.5
Epoch: 27 current acc: 0.5
Epoch: 28 current acc: 0.5
Epoch: 29 current acc: 0.5
Epoch: 30 current acc: 0.5
Epoch: 31 current acc: 0.75
Epoch: 32 current acc: 0.75
Epoch: 33 current acc: 0.75
Epoch: 34 current acc: 0.75
Epoch: 35 current acc: 0.75
Epoch: 36 current acc: 0.5
Epoch: 37 current acc: 0.5

65

In [26]:
epoch = 0
for i in range(100):
    nn = NN(4, pop_size=10)
    epoch += nn.neuro_train(print_details=False, crossover=True)

print(epoch/100)


Initial weights: 
W_hidden: [[0.91113748 0.63781629 0.87783647 0.02979478]
 [0.67535364 0.49483388 0.33704922 0.37127299]]
B_hidden: [[0.40530878 0.09841887 0.14743376 0.94428349]]
W_out: [[0.30621366]
 [0.07451656]
 [0.61880232]
 [0.66635015]]
B_out: [[0.79098883]]

Epoch: 53 current acc: 1.0
W_hidden: [[-1.38122376  0.88145109 -0.19076505 -0.68039595]
 [ 1.00219755  0.04995242  0.7046818   1.1525041 ]]
B_hidden: [[-0.06170582  0.40350638  0.59890422 -0.26469842]]
W_out: [[ 1.67839718]
 [ 0.63780664]
 [-0.53534177]
 [ 0.28548083]]
B_out: [[-0.47479489]]

Initial weights: 
W_hidden: [[0.6025833  0.1433239  0.77202723 0.91407303]
 [0.16747183 0.35049247 0.76047572 0.12367621]]
B_hidden: [[0.81414552 0.70668212 0.13958508 0.81679624]]
W_out: [[0.9155208 ]
 [0.72276481]
 [0.44442011]
 [0.83978415]]
B_out: [[0.8362212]]

Epoch: 41 current acc: 1.0
W_hidden: [[ 0.84446544  1.09512072  0.90627781  0.01031993]
 [ 1.14799936  0.2022778   0.4809016  -0.26866977]]
B_hidden: [[-0.22644731  0.0796

In [27]:
epoch = 0
for i in range(100):
    nn = NN(4, pop_size=10)
    epoch += nn.neuro_train(print_details=False, crossover=False)

print(epoch/100)

Initial weights: 
W_hidden: [[0.08444706 0.13925157 0.20277331 0.90200375]
 [0.07693366 0.112618   0.3950009  0.72629672]]
B_hidden: [[0.62159183 0.53643002 0.89232593 0.55415518]]
W_out: [[0.50895309]
 [0.41159765]
 [0.29310277]
 [0.11871626]]
B_out: [[0.48919886]]

Epoch: 41 current acc: 1.0
W_hidden: [[ 0.50084891  0.34116901  0.9838759   0.56493631]
 [ 0.76226977  1.25224021 -0.54470238 -0.52206407]]
B_hidden: [[ 0.94319619  1.21630652  1.30561665 -0.05989667]]
W_out: [[ 0.34279536]
 [-0.21430835]
 [-0.375419  ]
 [ 1.31885724]]
B_out: [[0.32672589]]

Initial weights: 
W_hidden: [[0.07029824 0.78450698 0.0189795  0.96288087]
 [0.63885866 0.2353767  0.80916321 0.63035371]]
B_hidden: [[0.38920094 0.56683604 0.03680958 0.08827925]]
W_out: [[0.71638206]
 [0.23195246]
 [0.71931634]
 [0.309573  ]]
B_out: [[0.32077048]]

Epoch: 47 current acc: 1.0
W_hidden: [[ 0.55296283  1.01732383 -0.64429924  0.31600839]
 [ 0.29976355  1.26631828  0.61621792  0.96618776]]
B_hidden: [[ 0.23781481 -0.9020

In [28]:
for i in nn.data:
    print(nn.execute(i))

[0.]
[1.]
[1.]
[0.]


Bei "Fitness Proportionate Selection" ist die Wahrscheinlichkeit eines jeden Individuums, selektiert zu werden, mit unserer Fitness-Funktion ziemlich klein. "Elitist Selection" bringt wiederum Konstanz in die Populationsgröße.

# Aufgabe 3


In [29]:
import tensorflow as tf

In [30]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2, 1)),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [31]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss_fn,
              metrics=['accuracy'])

In [32]:
data = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

target = np.array([[0],[1],[1],[0]])

In [33]:
model.fit(data, target, epochs=300)

Epoch 1/300
1/1 [==============================] - 1s 576ms/step - loss: 0.7330 - accuracy: 0.7500
Epoch 2/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7303 - accuracy: 0.7500
Epoch 3/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7282 - accuracy: 0.7500
Epoch 4/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7265 - accuracy: 0.7500
Epoch 5/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7252 - accuracy: 0.7500
Epoch 6/300
1/1 [==============================] - 0s 2ms/step - loss: 0.7238 - accuracy: 0.7500
Epoch 7/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7226 - accuracy: 0.7500
Epoch 8/300
1/1 [==============================] - 0s 4ms/step - loss: 0.7216 - accuracy: 0.7500
Epoch 9/300
1/1 [==============================] - 0s 4ms/step - loss: 0.7205 - accuracy: 0.7500
Epoch 10/300
1/1 [==============================] - 0s 3ms/step - loss: 0.7195 - accuracy: 0.7500
Epoch 11/300
1/1 [=========

In [34]:
model.evaluate(data,  target, verbose=2)

1/1 - 0s - loss: 0.6436 - accuracy: 1.0000


[0.6436121463775635, 1.0]

Von den drei Verfahren wird das Training bis hin zu einer Accuracy von 1 im Allgemeinen am schnellsten von dem Modell, das evolutionäre Algorithmen nutzt, durchgeführt. Am langsamsten ist die manuelle Variante. 